In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
### for reproducible experiments
from numpy.random import seed
seed(42)
from tensorflow.random import set_seed
set_seed(42)

/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
import numpy as np
import pandas as pd
import git
import mlflow.keras
import os, sys
from gensim.models.wrappers import FastText
import joblib
#import tensorflow_hub as hub
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#import random
%matplotlib inline
pd.set_option('display.max_rows', None)
from tqdm import tqdm
sys.path.append('../citation_bio_trainer')
#from feature.SpacyFeaturizer import get_spacy_feats_from_text
from feature_window.featurizer_window import Featurizer_window
from util.utils import calulate_ser_jer, load_from_folder, pad_sequences, load_embedding_matrix, evaluate, log_mlflow_results
import warnings
from model.ftlstm import calulate_ser_jer, get_model, plot_output
warnings.filterwarnings('ignore')

/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


## LSTM with random embedding model

In [4]:
%%time
train_data_path = '/nlp/cs_mixed_20k/cs_mixed_20k_train/'
test_data_path  = '/nlp/cs_mixed_20k/cs_mixed_20k_test/'
test_new_data_path = '/nlp/artificial_with_df_and_text/'
eval_without_intra_newline_path  = '/nlp/eval_data_spacy_tokenized_extra_space_removed/'

#sentences_train, sent_tags_train = load_from_folder(train_data_path)
sentences_test_new, sent_tags_test_new = load_from_folder(test_new_data_path)
sentences_eval1, sent_tags_eval1 = load_from_folder(eval_without_intra_newline_path)

CPU times: user 25.3 s, sys: 214 ms, total: 25.5 s
Wall time: 25.5 s


In [5]:
#xtrain, xvalid, ytrain, yvalid = train_test_split(sentences_train, sent_tags_train, test_size=0.1, random_state=42)

In [6]:
feat_config = {'max_vocab':100000,
               'lstm_feats':True, 
               'spacy_feats':True, 
               'google_feats': False, 
               'parscit_feats': False,
              'custom_feats': True, 
              'spacy_mode': 'production',
              'window':100,
              'step':90}

In [7]:
# %%time
#featurizer = Featurizer_window(feat_config)
#print("loading train ...")
#train_dict, tokenizer, maxlen = featurizer.fit_transform(xtrain, ytrain)
# print("loading valid ...")
# valid_dict = featurizer.transform(xvalid, yvalid)
# print("loading test ...")
# test_dict  = featurizer.transform(sentences_test, sent_tags_test)
# print("loading evals ...")
# eval1_dict  = featurizer.transform(sentences_eval1, sent_tags_eval1)

In [8]:
featurizer = joblib.load('featurizer.pkl')

In [9]:
%%time
test_new_dict  = featurizer.transform(sentences_test_new, sent_tags_test_new)
eval1_dict  = featurizer.transform(sentences_eval1, sent_tags_eval1)

CPU times: user 4min 22s, sys: 55.7 s, total: 5min 18s
Wall time: 5min 18s


In [10]:
#eval1_dict['lstm_feats'].shape

In [11]:
test_new_dict['lstm_feats'].shape

(30722, 100)

In [12]:
# import joblib
#joblib.dump(featurizer, 'featurizer.pkl')

## Keras with pre-trained fast text embedding

In [ ]:
#maxlen=100

In [13]:
%%time
#wiki_model = FastText.load_fasttext_format('/nlp/cc.en.300.bin')
#embedding_matrix = load_embedding_matrix(wiki_model, feat_config['max_vocab'], tokenizer.word_index, 300)
embedding_matrix=np.load('/nlp/temp/embedding_matrix.npy')

CPU times: user 2.83 ms, sys: 95.8 ms, total: 98.6 ms
Wall time: 98.1 ms


In [14]:
model_config = {'embedding': {'use':True, 'matrix':embedding_matrix, 'trainable':True},## trainableTrue is significantly better 
                'lstm': {'use':True, 'num': 1, 'units':50, 'dropout':0.2},
                'dense':{'use':False, 'num': 1, 'units':32, 'activation': 'relu', 'dropout':0.2},
                'optimizer': Adam(
        learning_rate=0.001,
        beta_1=0.0,
        beta_2=0.0,
        epsilon=1e-05,
        amsgrad=False,
    ),
                'output_activation' : 'sigmoid', 
                'batch_size': 32, ## lower the better
                'aux_feats': {'use':True, 'dim':17, 'place':'before_lstm'}, 
                'timedistributed':{'use':False}, ## no difference at all True or False
                'shuffle': True ## not much different than False
                
               }

In [ ]:
%%time
model = get_model(feat_config, model_config, maxlen)
print(model.summary())

In [15]:
%%time
if feat_config['spacy_feats'] and not feat_config['parscit_feats'] and not feat_config['custom_feats']:
    print('only spacy not parscit not custom')
    train_aux = train_dict['spacy_num_feats']
    valid_aux = valid_dict['spacy_num_feats']
    test_aux  = test_dict['spacy_num_feats']
    eval1_aux  = eval1_dict['spacy_num_feats']
    
elif feat_config['spacy_feats'] and  feat_config['parscit_feats'] and not feat_config['custom_feats']:
    print("both spacy and parscit not custom")
    train_aux = np.concatenate((train_dict['spacy_num_feats'], train_parscit_arr), axis=-1)
    valid_aux = np.concatenate((valid_dict['spacy_num_feats'], valid_parscit_arr), axis=-1)
    test_aux  = np.concatenate((test_dict['spacy_num_feats'], test_parscit_arr), axis=-1)
    eval1_aux  = np.concatenate((eval1_dict['spacy_num_feats'], eval1_parscit_arr), axis=-1)
    
elif feat_config['spacy_feats'] and  feat_config['custom_feats'] and not feat_config['parscit_feats']:
    print("both spacy and custom not parscit")
    #train_aux = np.concatenate((train_dict['spacy_num_feats'], train_dict['custom_feats']), axis=-1)
    #valid_aux = np.concatenate((valid_dict['spacy_num_feats'], valid_dict['custom_feats']), axis=-1)
    #test_aux  = np.concatenate((test_dict['spacy_num_feats'], test_dict['custom_feats']), axis=-1)
    eval1_aux  = np.concatenate((eval1_dict['spacy_num_feats'], eval1_dict['custom_feats']), axis=-1)
    test_new_aux  = np.concatenate((test_new_dict['spacy_num_feats'], test_new_dict['custom_feats']), axis=-1)

elif feat_config['spacy_feats'] and  feat_config['parscit_feats'] and feat_config['custom_feats']:
    print("all spacy and parscit and custom features")
    train_aux = np.concatenate((train_dict['spacy_num_feats'], train_parscit_arr, train_custom_arr), axis=-1)
    valid_aux = np.concatenate((valid_dict['spacy_num_feats'], valid_parscit_arr, valid_custom_arr), axis=-1)
    test_aux  = np.concatenate((test_dict['spacy_num_feats'], test_parscit_arr, test_custom_arr), axis=-1)
    eval1_aux  = np.concatenate((eval1_dict['spacy_num_feats'], eval1_parscit_arr, eval1_custom_arr), axis=-1)
else:
    print('only lstm features')

both spacy and custom not parscit
CPU times: user 262 ms, sys: 176 ms, total: 438 ms
Wall time: 437 ms


In [ ]:
%%time
early_stop= EarlyStopping(monitor='val_loss',patience=3,verbose=0,mode='min',restore_best_weights=False, min_delta=0.0001)

## multi input
if model_config['aux_feats']['use']:
    print("multi input model")
    history = model.fit([train_dict['lstm_feats'], train_aux], train_dict['labels'], verbose=1, epochs=100, batch_size= model_config['batch_size'], \
                    validation_data=([valid_dict['lstm_feats'], valid_aux], valid_dict['labels']), callbacks=[early_stop], shuffle=model_config['shuffle'])
else:
    print("single input model")
    history = model.fit(train_dict['lstm_feats'], train_dict['labels'], verbose=1, epochs=100, batch_size= model_config['batch_size'], \
                        validation_data=(valid_dict['lstm_feats'], valid_dict['labels']), callbacks=[early_stop], shuffle=model_config['shuffle'])

plot_output(history)

In [1]:
print('done')

done


In [32]:
print('test')

test


In [ ]:
#sentences_test[0]

### Analysis on validation data

In [ ]:
%%time
#keras_model = mlflow.keras.load_model("s3://caps-s3-mlflow/artifacts/4/59e06c5ff45944f5bf56745b2e19cb05/artifacts/models")
if model_config['aux_feats']['use']:
    valid_probs = model.predict([valid_dict['lstm_feats'], valid_aux])
else:
    valid_probs = model.predict(valid_dict['lstm_feats'])

valid_probs = valid_probs.reshape(valid_probs.shape[0], valid_probs.shape[1])
valid_preds = np.where(valid_probs > 0.5, 1, 0)

valid_true_ls = [i[0:len(j)].tolist() for i,j in zip(valid_dict['labels'], valid_dict['tags_window'])]
valid_pred_ls = [i[0:len(j)].tolist() for i,j in zip(valid_preds, valid_dict['tags_window'])]
result_valid = evaluate(valid_true_ls, valid_pred_ls)

In [33]:
result_valid

{'count': 155569,
 'count_citations': 197153,
 'mean_ser': 0.07548007585466847,
 'mean_jer': 0.07745027547108939,
 'mean_acc': 0.9997166608829646,
 'num_mistakes_seq': 2097,
 'num_mistakes_all': 2199,
 'mistakes_per_seq': 1.0486409155937053,
 'perc_mistakes_seq': 1.3479549267527593,
 'perc_mistake_per_citation': 1.1153773972498517}

In [26]:
result_valid

{'count': 18654,
 'count_citations': 45668,
 'mean_ser': 0.035972241367866975,
 'mean_jer': 0.03794619988733852,
 'mean_acc': 0.9998070034769974,
 'num_mistakes_seq': 274,
 'num_mistakes_all': 299,
 'mistakes_per_seq': 1.0912408759124088,
 'perc_mistakes_seq': 1.4688538651227618,
 'perc_mistake_per_citation': 0.6547254094770956}

In [ ]:
#keras_model = mlflow.keras.load_model("s3://caps-s3-mlflow/artifacts/4/977e2cc7b36040f79dde3d6303d26952/artifacts/models")
count_valid = 0
#valid_df = pd.DataFrame([])
valid_ls = []
for ind in range(len(valid_preds)):
    temp_ls = []
    pred = valid_preds[ind]
    true = valid_dict['labels'][ind]
    if (true == pred).all():
        pass
    else:
        count_valid += 1
        fp_ind = np.where((pred == 1) & (true == 0))[0]
        fn_ind = np.where((pred == 0) & (true == 1))[0]
        if len(fp_ind) > 0:
            for x in fp_ind:
                valid_ls.append([ind, 'FP'] + np.array(xvalid[ind].split(" "))[max(0, x-3):x+4].tolist())
                #print(ind, 'FP', np.array(xvalid[ind].split(" "))[max(0, x-3):x+4])
        if len(fn_ind) > 0:
            #print(ind, 'False negatives:', fn_ind)
            for x in fn_ind:
                valid_ls.append([ind, 'FN'] + np.array(xvalid[ind].split(" "))[max(0, x-3):x+4].tolist())
                #print(ind, 'FN', np.array(xvalid[ind].split(" "))[max(0, x-3):x+4])

In [ ]:
valid_mistakes_df = pd.DataFrame(valid_ls, columns=['index', 'error_type', 'x-3', 'x-2', 'x-1', 'x', 'x+1', 'x+2', 'x+3'])

### Analysis of test result

In [16]:
%%time
##### CHANGE THE KERAS MODEL LOADING PART ******************************************
model = mlflow.keras.load_model("s3://caps-s3-mlflow/artifacts/4/aea7f13e822d408e827726c2b972b93c/artifacts/models/")
if model_config['aux_feats']['use']:
    test_probs = model.predict([test_new_dict['lstm_feats'], test_new_aux])
else:
    test_probs = model.predict(test_new_dict['lstm_feats'])
test_probs = test_probs.reshape(test_probs.shape[0], test_probs.shape[1])
test_preds = np.where(test_probs > 0.5, 1, 0)

CPU times: user 12.5 s, sys: 3.6 s, total: 16.1 s
Wall time: 27 s


In [17]:
%%time
take_mean = True
test_probs_win = []
for ind in range(len(test_probs)):
    #print(ind)
    test_probs_win.append(test_probs[ind][0:len(test_new_dict['sentences_window'][ind])])

test_probs_merged = []
window = 100
step = 90
gind = 0
for bigind in range(len(sentences_test_new)):
    temp = []
    seqlen =  len(sentences_test_new[bigind])
    iter = 0
    #print(seqlen)
    #print(bigind)
    for smallind in range(gind, len(test_probs_win)):
        if len(temp) < seqlen:
            if iter ==0:
                temp+=test_probs_win[gind].tolist()
                iter +=1
                gind += 1
            else:
                if take_mean:
                    temp[-(window-step):] = np.mean([np.array(temp[-(window-step):]), test_probs_win[gind][0:(window-step)]], axis=0).tolist()
                    temp += test_probs_win[gind][(window-step):].tolist()
                    gind += 1
                else:
                    temp+=test_probs_win[gind][(window-step):].tolist()
                    gind += 1
        else:
            break
    test_probs_merged.append(temp)

tag2index = {'B-CIT':1, 'I-CIT':0}
test_preds_merged = [[int(t>0.5) for t in test_probs_merged[ind]] for ind in range(len(test_probs_merged))]
test_true_merged = [[tag2index[t] for t in sent_tags_test_new[ind]] for ind in range(len(sent_tags_test_new))]

CPU times: user 1.78 s, sys: 155 ms, total: 1.93 s
Wall time: 1.83 s


In [18]:
%%time
result_test = evaluate(test_true_merged, test_preds_merged)

CPU times: user 10.1 s, sys: 0 ns, total: 10.1 s
Wall time: 10.1 s


In [19]:
result_test

{'count': 1458,
 'count_citations': 68542,
 'mean_ser': 0.00018065941575368805,
 'mean_jer': 0.0009199697553736244,
 'mean_acc': 0.9999717024811379,
 'num_mistakes_seq': 77,
 'num_mistakes_all': 81,
 'mistakes_per_seq': 1.051948051948052,
 'perc_mistakes_seq': 5.2812071330589845,
 'perc_mistake_per_citation': 0.11817571707857956}

In [30]:
result_test

{'count': 1953,
 'count_citations': 46092,
 'mean_ser': 0.0007581633612078364,
 'mean_jer': 0.002062706327176513,
 'mean_acc': 0.999909105171757,
 'num_mistakes_seq': 93,
 'num_mistakes_all': 121,
 'mistakes_per_seq': 1.3010752688172043,
 'perc_mistakes_seq': 4.761904761904762,
 'perc_mistake_per_citation': 0.26251844137811337}

In [20]:
%%time
tags = {'dataset':'cs_mixed_20k', 'data_split':'test_new', 'version':2} 
del model_config['embedding']['matrix'] 
opt = model_config['optimizer']
model_config['optimizer'] = str(opt.get_config())
log_mlflow_results(model, result_test, feat_config, model_config, tags)

2020/09/18 16:42:52 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under s3://caps-s3-mlflow/artifacts/4/76e237711b604a039e3bc052b5d4ac98/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.


CPU times: user 5.08 s, sys: 1.65 s, total: 6.72 s
Wall time: 11.9 s


In [30]:
count_test = 0
test_ls = []
for ind in range(len(test_preds_merged)):
    temp_ls = []
    pred = np.array(test_preds_merged[ind])
    true = np.array(test_true_merged[ind])
    if (true == pred).all():
        pass
    else:
        count_test += 1
        fp_ind = np.where((pred == 1) & (true == 0))[0]
        fn_ind = np.where((pred == 0) & (true == 1))[0]
        if len(fp_ind) > 0:
            #print(ind, 'fp', fp_ind)
            for x in fp_ind:
                ls = ['None']*7
                if max(0, x-3)==0:
                    ls[3-x:] = np.array(sentences_test[ind].split(" "))[max(0, x-3):x+4].tolist()
                else:
                    ls = np.array(sentences_test[ind].split(" "))[max(0, x-3):x+4].tolist()
                test_ls.append([ind, x, 'FP'] + ls)
                #print(ind, 'FP', np.array(sentences_test[ind].split(" "))[max(0, x-3):x+4])
        if len(fn_ind) > 0:
            #print(ind, 'False negatives:', fn_ind)
            #print(ind, 'fn', fn_ind)
            for x in fn_ind:
                ls = ['None']*7
                if max(0, x-3)==0:
                    ls[3-x:] = np.array(sentences_test[ind].split(" "))[max(0, x-3):x+4].tolist()
                else:
                    ls = np.array(sentences_test[ind].split(" "))[max(0, x-3):x+4].tolist()
                test_ls.append([ind, x, 'FN'] + ls)
                #print(ind, 'FN', np.array(sentences_test[ind].split(" "))[max(0, x-3):x+4])

In [31]:
test_mistakes_df = pd.DataFrame(test_ls, columns=['seq_index', 'token_index', 'error_type', 'x-3', 'x-2', 'x-1', 'x', 'x+1', 'x+2', 'x+3'])

In [32]:
test_mistakes_df = test_mistakes_df.replace('\n', 'newline')

In [ ]:
#test_mistakes_df[test_mistakes_df.error_type == 'FN']

In [35]:
test_mistakes_df.error_type.value_counts()

FN    86
FP    35
Name: error_type, dtype: int64

In [58]:
test_mistakes_df.to_csv('../test_mistakes_windownew.csv')

In [ ]:
#test_mistakes_df[test_mistakes_df.error_type == 'FP']

### Analysis of eval dataset

In [ ]:
%%time
#keras_model = mlflow.keras.load_model("s3://caps-s3-mlflow/artifacts/4/1d953e89279b49fdb63cd7b3cb6c8b0b/artifacts/models")

In [2]:
model = mlflow.keras.load_model("s3://caps-s3-mlflow/artifacts/4/aea7f13e822d408e827726c2b972b93c/artifacts/models/")
if model_config['aux_feats']['use']:
    print('auxiliary features ...')
    eval1_probs = model.predict([eval1_dict['lstm_feats'], eval1_aux])
else:
    print('only lstm features ...')
    eval1_probs = model.predict(eval1_dict['lstm_feats'])
eval1_probs = eval1_probs.reshape(eval1_probs.shape[0], eval1_probs.shape[1])
eval1_preds = np.where(eval1_probs > 0.5, 1, 0)

NameError: name 'mlflow' is not defined

In [38]:
%%time
take_mean = True
eval1_probs_win = []
for ind in range(len(eval1_probs)):
    #print(ind)
    eval1_probs_win.append(eval1_probs[ind][0:len(eval1_dict['sentences_window'][ind])])

eval1_probs_merged = []
window = 100
step = 90
gind = 0
for bigind in range(len(sentences_eval1)):
    temp = []
    seqlen =  len(sentences_eval1[bigind].split(" "))
    iter = 0
    #print(seqlen)
    #print(bigind)
    for smallind in range(gind, len(eval1_probs_win)):
        if len(temp) < seqlen:
            if iter ==0:
                temp+=eval1_probs_win[gind].tolist()
                iter +=1
                gind += 1
            else:
                if take_mean:
                    temp[-(window-step):] = np.mean([np.array(temp[-(window-step):]), eval1_probs_win[gind][0:(window-step)]], axis=0).tolist()
                    temp += eval1_probs_win[gind][(window-step):].tolist()
                    gind += 1
                else:
                    temp+=eval1_probs_win[gind][(window-step):].tolist()
                    gind += 1
        else:
            break
    eval1_probs_merged.append(temp)

tag2index = {'B-CIT':1, 'I-CIT':0}
eval1_preds_merged = [[int(t>0.5) for t in eval1_probs_merged[ind]] for ind in range(len(eval1_probs_merged))]
eval1_true_merged = [[tag2index[t] for t in sent_tags_eval1[ind].split(" ")] for ind in range(len(sent_tags_eval1))]

CPU times: user 1.83 s, sys: 104 ms, total: 1.94 s
Wall time: 1.93 s


In [39]:
result_eval1 = evaluate(eval1_true_merged, eval1_preds_merged)

In [35]:
result_eval1

{'count': 1328,
 'count_citations': 56850,
 'mean_ser': 1.2982966348151225e-05,
 'mean_jer': 0.00041063361934943446,
 'mean_acc': 0.9999894054432367,
 'num_mistakes_seq': 18,
 'num_mistakes_all': 26,
 'mistakes_per_seq': 1.4444444444444444,
 'perc_mistakes_seq': 1.355421686746988,
 'perc_mistake_per_citation': 0.04573438874230431}

In [40]:
result_eval1

{'count': 1328,
 'count_citations': 56850,
 'mean_ser': 0.0007642510339866931,
 'mean_jer': 0.0075459564576609486,
 'mean_acc': 0.9998296369743739,
 'num_mistakes_seq': 353,
 'num_mistakes_all': 411,
 'mistakes_per_seq': 1.1643059490084986,
 'perc_mistakes_seq': 26.58132530120482,
 'perc_mistake_per_citation': 0.7229551451187335}

In [105]:
result_eval1

{'count': 1328,
 'count_citations': 56850,
 'mean_ser': 0.0007776320593378298,
 'mean_jer': 0.008467544104861686,
 'mean_acc': 0.9998095060436722,
 'num_mistakes_seq': 422,
 'num_mistakes_all': 463,
 'mistakes_per_seq': 1.0971563981042654,
 'perc_mistakes_seq': 31.77710843373494,
 'perc_mistake_per_citation': 0.8144239226033422}

In [30]:
result_eval1

{'count': 1328,
 'count_citations': 56850,
 'mean_ser': 0.0007637693631669535,
 'mean_jer': 0.006231187644864529,
 'mean_acc': 0.9998530866974515,
 'num_mistakes_seq': 300,
 'num_mistakes_all': 344,
 'mistakes_per_seq': 1.1466666666666667,
 'perc_mistakes_seq': 22.59036144578313,
 'mistake_per_citation': 0.6051011433597185}

### Visualize errors

In [41]:
count_eval1 = 0
eval1_ls = []
for ind in range(len(eval1_preds_merged)):
    temp_ls = []
    pred = np.array(eval1_preds_merged[ind])
    true = np.array(eval1_true_merged[ind])
    if (true == pred).all():
        pass
    else:
        count_eval1 += 1
        fp_ind = np.where((pred == 1) & (true == 0))[0]
        fn_ind = np.where((pred == 0) & (true == 1))[0]
        if len(fp_ind) > 0:
            #print(ind, 'fp', fp_ind)
            for x in fp_ind:
                ls = ['None']*7
                if max(0, x-3)==0:
                    ls[3-x:] = np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4].tolist()
                else:
                    ls = np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4].tolist()
                eval1_ls.append([ind, x, 'FP'] + ls)
                #print(ind, 'FP', np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4])
        if len(fn_ind) > 0:
            #print(ind, 'False negatives:', fn_ind)
            #print(ind, 'fn', fn_ind)
            for x in fn_ind:
                ls = ['None']*7
                if max(0, x-3)==0:
                    ls[3-x:] = np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4].tolist()
                else:
                    ls = np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4].tolist()
                eval1_ls.append([ind, x, 'FN'] + ls)
                #print(ind, 'FN', np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4])

In [42]:
#eval1_ls

In [43]:
eval_mistakes_df = pd.DataFrame(eval1_ls, columns=['seq_index', 'token_index', 'error_type', 'x-3', 'x-2', 'x-1', 'x', 'x+1', 'x+2', 'x+3'])

In [44]:
eval_mistakes_df = eval_mistakes_df.replace('\n', 'newline')

In [71]:
eval_mistakes_df

,seq_index,token_index,error_type,x-3,x-2,x-1,x,x+1,x+2,x+3
0,171,420,FP,",",Ekpa,newline,",",Cooper,TK,","
1,210,772,FN,),.,newline,18,Hunger,-,Craig
2,371,1286,FN,34,.,newline,.,A,Consensus,on
3,412,251,FN,76,.,newline,[,8]M.,Preene,","
4,412,561,FN,4260,.,newline,[,17]V.Hamm,",",B.B.Sabet
5,706,828,FN,after,subacute,newline,Monica,",",F.Z.,","
6,712,2833,FN,346−352,.,newline,66,.,http://www.ozm.cz/en/sensitivity-tests/small-s...,newline
7,776,282,FN,257,.,newline,[,10]J.W.Xi,",",Control
8,776,416,FN,129,.,newline,[,14]X.X.Zhao,",",L.J.Li
9,841,1458,FN,remodeling,in,newline,Saccharomyces,cerevisiae,mutants,affected


In [56]:
eval_mistakes_df.to_csv('../eval_mistakes_windownew.csv', index=False)

### Save eval results to file

In [ ]:
ind = 0
eval_path_dict = {}
for fpath in os.listdir(eval_without_intra_newline_path):
    if fpath not in ['data-gen-config.json', 'data_generation_stats.csv'] and ".csv" in fpath:
        eval_path_dict[ind] = fpath
        ind += 1

In [ ]:
eval_folder_path = '/nlp/eval_data_retok_predictions_f3a9cc61e30f460499b95a4c2b7957ae'
if not os.path.exists(eval_folder_path):
    os.makedirs(eval_folder_path)
for ind in range(len(eval_preds1)):
    #print(ind)
    df = pd.DataFrame([], columns=['x', 'y'])
    seq_len = min(len(sentences_eval1[ind].split(" ")), maxlen)
    df['x'] = sentences_eval1[ind].split(" ")[0:seq_len]
    df['y'] = eval_preds1[ind][0:seq_len]
    df.to_csv(os.path.join(eval_folder_path, eval_path_dict[ind][0:-4] + '_pred.csv'))

### Log Eval results to MLFlow

In [36]:
tags = {'dataset':'cs_mixed_20k', 'data_split':'eval_with_intra_newline_path','version':2}
#del model_config['embedding']['matrix'] 
#opt = model_config['optimizer']
#model_config['optimizer'] = str(opt.get_config())
log_mlflow_results(model, result_eval1, feat_config, model_config, tags)

2020/09/06 16:20:20 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under s3://caps-s3-mlflow/artifacts/4/aea7f13e822d408e827726c2b972b93c/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.
